In [15]:
from pathlib import Path

import torch
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder

DATASET_PATH = Path("datasets")
TRAIN_PATH = DATASET_PATH / "train"
TEST_PATH = DATASET_PATH / "test"
VAL_PATH = DATASET_PATH / "val"
img_size = 128
train_ds = ImageFolder(
    TRAIN_PATH.as_posix(),
    transforms.Compose([
        transforms.Resize([img_size, img_size]),
        # transforms.RandomHorizontalFlip(),
        # transforms.RandomRotation(15),
        # transforms.RandomAffine(15),
        # transforms.GaussianBlur(5, (0.1, 0.5)),
        # transforms.ColorJitter(0.2, 0.2, 0.1, 0.01),
        # transforms.CenterCrop(img_size),
        transforms.ToTensor(),
    ]))
train_dl = DataLoader(train_ds)
val_ds = ImageFolder(
    VAL_PATH.as_posix(),
    transforms.Compose([
        transforms.Resize([img_size, img_size]),
        # transforms.CenterCrop(img_size),
        transforms.ToTensor(),
    ])
)
val_dl = DataLoader(val_ds)

In [16]:
# Create a CNN Module
import torch
import torch.nn as nn
import torch.nn.functional as F


class CNNBlock(nn.Module):
    def __init__(self, c_in, c_out, kernel=3, ave_pool=True):
        super(CNNBlock, self).__init__()

        self.net = nn.Sequential(
            nn.Conv2d(c_in, c_out, kernel, 1, int((kernel - 1) // 2)),
            nn.BatchNorm2d(c_out),
            nn.ReLU(inplace=True),

        )
        if ave_pool:
            self.net.append(nn.AvgPool2d(2, 2))

    def forward(self, x):
        return self.net(x)


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.net = nn.Sequential(
            CNNBlock(3, 16, 7, ave_pool=False),
            CNNBlock(16, 16, 5),
            CNNBlock(16, 32),
            CNNBlock(32, 32, ave_pool=False),
            CNNBlock(32, 64),
            CNNBlock(64, 64, ave_pool=False),
            CNNBlock(64, 128),
            CNNBlock(128, 128, ave_pool=False),
            CNNBlock(128, 128),
            CNNBlock(128, 128, ave_pool=False),
            CNNBlock(128, 128),
            CNNBlock(128, 128, ave_pool=False),
            CNNBlock(128, 128),
            nn.Flatten(),
            nn.Linear(128, 2),
            nn.Dropout(0.1),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        return self.net(x)


CNN()(torch.rand([5, 3, img_size, img_size]))

tensor([[0.5124, 0.4876],
        [0.4171, 0.5829],
        [0.4238, 0.5762],
        [0.4906, 0.5094],
        [0.5132, 0.4868]], grad_fn=<SoftmaxBackward0>)

In [17]:
from torchinfo import summary
summary(CNN())

Layer (type:depth-idx)                   Param #
CNN                                      --
├─Sequential: 1-1                        --
│    └─CNNBlock: 2-1                     --
│    │    └─Sequential: 3-1              2,400
│    └─CNNBlock: 2-2                     --
│    │    └─Sequential: 3-2              6,448
│    └─CNNBlock: 2-3                     --
│    │    └─Sequential: 3-3              4,704
│    └─CNNBlock: 2-4                     --
│    │    └─Sequential: 3-4              9,312
│    └─CNNBlock: 2-5                     --
│    │    └─Sequential: 3-5              18,624
│    └─CNNBlock: 2-6                     --
│    │    └─Sequential: 3-6              37,056
│    └─CNNBlock: 2-7                     --
│    │    └─Sequential: 3-7              74,112
│    └─CNNBlock: 2-8                     --
│    │    └─Sequential: 3-8              147,840
│    └─CNNBlock: 2-9                     --
│    │    └─Sequential: 3-9              147,840
│    └─CNNBlock: 2-10                

In [18]:
# Create a pytorch lightning module
import pytorch_lightning as pl


class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = CNN()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        acc = torch.sum(torch.argmax(y_hat, dim=1) == y) / y.shape[0]
        self.log("train_acc", acc)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        acc = torch.sum(torch.argmax(y_hat, dim=1) == y) / y.shape[0]
        self.log("acc", acc)

    def configure_optimizers(self):
        optim = torch.optim.Adam(self.parameters(), lr=0.0005, weight_decay=0.001)

        return {
            "optimizer": optim,
            "lr_scheduler": {
                "scheduler": ReduceLROnPlateau(optim, mode='max', factor=0.2, patience=2, verbose=True),
                "monitor": "val_mae",
            },
        }


In [19]:
model = LitModel()

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping

early_stop_callback = EarlyStopping(monitor="acc", min_delta=0.00, patience=4, verbose=False, mode="max")
trainer = pl.Trainer(max_epochs=40,
                     accelerator='gpu',
                     callbacks=[early_stop_callback]
                     )
trainer.fit(
    model,
    train_dataloaders=DataLoader(train_ds, batch_size=64, shuffle=True, num_workers=6),
    val_dataloaders=DataLoader(val_ds, batch_size=128, shuffle=False, num_workers=2)
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | CNN  | 1.0 M 
-------------------------------
1.0 M     Trainable params
0         Non-trainable params
1.0 M     Total params
4.160     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\johnc\anaconda3\envs\IE4483\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

# Notes
- Batch Size greatly speeds up convergence
- Learning Rate has *some* effect on reducing jitter during training
- Adding more 3x3 2DConv seems to consistently improve the network
- Adding weight decay regularization for Adam reduces training overfitting
- Adding BatchNorm significantly improves performance
- ReLU performs significantly better than Sigmoid
- Having a tiny Fully Connected Layer at the end improves training stability

# Changes
- v11 uses a tiny FCN at the end
- v11 -> v12 uses a 2048 -> DropOut -> 1024 -> DropOut -> 2 instead of 2048 -> 2
- v11 -> v13 uses a AvgPool at the end (mimicking resnet)
- v11 -> v14 uses LeakyReLU (Not much difference)
- v11 -> v15 uses increase image size slightly, cropping out the middle
- v15 -> v16 move crop to end of transform pipeline
- v20 AvePoolModel

# Changes
- v11 uses a tiny FCN at the end
- v11 -> v12 uses a 2048 -> DropOut -> 1024 -> DropOut -> 2 instead of 2048 -> 2
- v11 -> v13 uses a AvgPool at the end (mimicking resnet)
- v11 -> v14 uses LeakyReLU (Not much difference)
- v11 -> v15 uses increase image size slightly, cropping out the middle
- v15 -> v16 move crop to end of transform pipeline
- v20 AvePoolModel

In [5]:


tor.mos